In [1]:
import subprocess

In [2]:
AMBERHOME = '/home/ali/Amber/amber22'
%cd mmpbsa

/home/ali/calstate/GBNN/GBNSR6_py/mmpbsa


In [5]:
def get_numbers(line, limit = -1):
    new_digit = ''
    numbers = []
    is_neg = False
    for i in range(len(line)):
        c = line[i]
        if i < len(line) - 1 and c == '-' and line[i + 1].isdigit():
            is_neg = True
        if c.isdigit() or (c == '.' and new_digit != ''):
            new_digit += c
        if new_digit != '' and c != '.' and (i == len(line) - 1 or not c.isdigit()):
            new_num = float(new_digit)
            new_num = -new_num if is_neg else new_num
            numbers.append(new_num)
            if len(numbers) == limit:
                return numbers
            new_digit = ''
    return numbers

In [4]:
def change_inp_endframe(i):
    new_lines=[]
    for l in open('mmpbsa.in'):
        if l.__contains__('endframe'):
            new_lines.append('   endframe=' + str(i) + ',\n')
        else:
            new_lines.append(l)
    f = open('mmpbsa.in', 'w')
    for l in new_lines:
        f.write(l)
    f.close()

In [5]:
full_record = []

In [1]:
f = open('F.dat').readlines()

In [2]:
pb = ''.join(f).split('POISSON BOLTZMANN:')[1].split('Differences (Complex - Receptor - Ligand):')[1]

In [6]:
mmpbsa_dic = {}
for param in ['EEL', 'EPB', 'ENPOLAR', 'EDISPER']:
    mmpbsa_dic[param] = get_numbers(pb.split('DELTA G gas')[0].split(param)[1].split('\n')[0])[0]

In [7]:
mmpbsa_dic

{'EEL': -959.173, 'EPB': 947.1114, 'ENPOLAR': -50.9787, 'EDISPER': 101.1385}

In [6]:
mmpbsa_dic = {}
for i in range(1, 20):
    print("End frame is " + str(i))
    change_inp_endframe(i)
    subprocess.run(['./run.sh'])
    f = open('FINAL_RESULTS_MMPBSA.dat').readlines()
    gb = ''.join(f).split('GENERALIZED BORN:')[1].split('POISSON BOLTZMANN:')[0]
    mmpbsa_dic = {}
    for title in ['Complex', 'Receptor', 'Ligand']:
        for param in ['EEL', 'EGB', 'ESURF']:
            mmpbsa_dic[title + '_' + param] = get_numbers(gb.split(title)[1].split('TOTAL')[0].split(param)[1].split('\n')[0])[0]
    full_record.append(mmpbsa_dic)

End frame is 1
End frame is 2
End frame is 3
End frame is 4
End frame is 5
End frame is 6
End frame is 7
End frame is 8
End frame is 9
End frame is 10
End frame is 11
End frame is 12
End frame is 13
End frame is 14
End frame is 15
End frame is 16
End frame is 17
End frame is 18
End frame is 19


In [8]:
import pickle
with open('prod1_1_20_mmpbsa.pckl', 'wb') as handle:
    pickle.dump(full_record, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
full_record[0]

{'Complex_EEL': -17231.1931,
 'Complex_EGB': -3313.2849,
 'Complex_ESURF': 86.814,
 'Receptor_EEL': -11572.6879,
 'Receptor_EGB': -2584.014,
 'Receptor_ESURF': 60.8072,
 'Ligand_EEL': -4670.1313,
 'Ligand_EGB': -1694.3062,
 'Ligand_ESURF': 36.9751}